In [1]:
!pip install transformers

In [2]:
!pip install spacy

In [3]:
!pip install bert-extractive-summarizer

In [4]:
!pip install torch torchvision torchaudio

In [5]:
!pip install sentencepiece

In [6]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

from summarizer import Summarizer,TransformerSummarizer
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [11]:
df = pd.read_csv('../../../downloads/water_problem_nlp_en_for_Kaggle_100.csv.xls', delimiter=';', header=0)
df = df.fillna(0)

convert_dict = {'text': str, 
                'env_problems': int,
                'pollution': int, 
                'treatment': int,
                'climate': int,
                'biomonitoring': int} 
  
df = df.astype(convert_dict)
df = df[:5]
df

,text,env_problems,pollution,treatment,climate,biomonitoring
0,In the Southern Bug basin wastewater is discha...,1,1,0,0,0
1,This amount is distributed as follows: urban a...,1,1,0,0,0
2,Despite the similar volumes of discharged wast...,1,1,0,0,0
3,They bring 84% of organic pollution 86% of pho...,1,1,0,0,0
4,The input of the industry is between 7–21% and...,1,1,0,0,0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text           5 non-null      object
 1   env_problems   5 non-null      int64 
 2   pollution      5 non-null      int64 
 3   treatment      5 non-null      int64 
 4   climate        5 non-null      int64 
 5   biomonitoring  5 non-null      int64 
dtypes: int64(5), object(1)
memory usage: 368.0+ bytes


In [13]:
df['text'].head(10)

0    In the Southern Bug basin wastewater is discha...
1    This amount is distributed as follows: urban a...
2    Despite the similar volumes of discharged wast...
3    They bring 84% of organic pollution 86% of pho...
4    The input of the industry is between 7–21% and...
Name: text, dtype: object

In [14]:
df['text'].str.len().max()

291

In [15]:
max_length = 400  # minimum characters in each block
i = 0
bodies = []
while i < len(df):
    body = ""
    body_empty = True
    while (len(body) < max_length) and (i < len(df)):
        if body_empty:
            body = df.loc[i,'text']
            body_empty = False
        else: body += " " + df.loc[i,'text']
        i += 1
    bodies.append(body)
    print("Length of blocks =", len(body))
print(f"\nNumber of text blocks = {len(bodies)}\n")
print("Text blocks:\n", bodies)

Length of blocks = 410
Length of blocks = 553

Number of text blocks = 2

Text blocks:
 ['In the Southern Bug basin wastewater is discharged from 341 organized sources (Map 5). In 2011 (reference year) 211 mln. m3 was discharged in total. This amount is distributed as follows: urban and rural housing wastewater treatment facilities (WWTPs - vodocanals) – 70 mln. m3 (57 users) industry – 50 mln. m3 (79 users) agriculture – 90 mln. m3 (193 users) other users – 1 mln. m3 (12 users) (Picture. 2.1). ', 'Despite the similar volumes of discharged wastewater major part of pollutants comes with communal WWTPs. They bring 84% of organic pollution 86% of phosphate ions and 84% of mineral nitrogen 91% of ammonia nitrogen 87% nitrate nitrogen and 79% nitrite nitrogen. The input of the industry is between 7–21% and agriculture has the lowest impact on water bodies - 0–6%. Of the 92 urban areas only 51 localities (55%) have centralized collection of communal waste waters and their monitoring. Among t

In [16]:
min_length_text = 40

In [17]:
xlnet_summary = []
for i in range(len(bodies)):
    model = TransformerSummarizer(transformer_type="XLNet",transformer_model_key="xlnet-base-cased")
    xlnet_summary.append(''.join(model(bodies[i], min_length=min_length_text)))

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
for i in range(len(bodies)):
    print("\nXLNet Summarizing Result:")
    print(xlnet_summary[i])


XLNet Summarizing Result:
In the Southern Bug basin wastewater is discharged from 341 organized sources (Map 5). This amount is distributed as follows: urban and rural housing wastewater treatment facilities (WWTPs - vodocanals) – 70 mln.

XLNet Summarizing Result:
Despite the similar volumes of discharged wastewater major part of pollutants comes with communal WWTPs. Of the 92 urban areas only 51 localities (55%) have centralized collection of communal waste waters and their monitoring.
